# 3w preparing

## pre

1. regularization : 소문자/대문자 변환 : 전체에서
4. regularization : 약어 대체 : 전체에서 
1. tokenization : 텍스트 -> 문장
2. tokenization : 문자 -> 단어
1. regularization : 문장 부호 제거 : 단어 단위에서
3. regularization : 불용어 처리 : 단어 단위에서
5. regularization : 반복되는 문자 처리 : 단어 단위에서

## 작업 순서

- num to word : don't do that, cuz' too many
- stop words control
- save to np array

In [1]:
import pandas as pd
import numpy as np
import os, sys, warnings
from tqdm import tqdm_notebook as tqdm
import re, string
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords

warnings.filterwarnings('ignore')
sys.path.append(os.path.abspath(os.path.dirname('../../modules/')))
from replacers import RegexpReplacer, RepeatReplacer, WordReplacer

In [2]:
path_github = '../../dataset/github/'
os.listdir(path_github)

['constants.py',
 'CONTRIBUTING.md',
 'gap-development.tsv',
 'gap-test.tsv',
 'gap-validation.tsv',
 'gap_scorer.py',
 'LICENSE',
 'README.md']

In [3]:
test = pd.read_csv(path_github+'gap-test.tsv', sep='\t')
validation = pd.read_csv(path_github+'gap-validation.tsv', sep='\t')
train = pd.read_csv(path_github+'gap-development.tsv', sep='\t')

In [4]:
validation.head()

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,URL
0,validation-1,He admitted making four trips to China and pla...,him,256,Jose de Venecia Jr,208,False,Abalos,241,False,http://en.wikipedia.org/wiki/Commission_on_Ele...
1,validation-2,"Kathleen Nott was born in Camberwell, London. ...",She,185,Ellen,110,False,Kathleen,150,True,http://en.wikipedia.org/wiki/Kathleen_Nott
2,validation-3,"When she returns to her hotel room, a Liberian...",his,435,Jason Scott Lee,383,False,Danny,406,True,http://en.wikipedia.org/wiki/Hawaii_Five-0_(20...
3,validation-4,"On 19 March 2007, during a campaign appearance...",he,333,Reucassel,300,True,Debnam,325,False,http://en.wikipedia.org/wiki/Craig_Reucassel
4,validation-5,"By this time, Karen Blixen had separated from ...",she,427,Finch Hatton,290,False,Beryl Markham,328,True,http://en.wikipedia.org/wiki/Denys_Finch_Hatton


In [5]:
# 대/소문자
def lower_casing(x):
    return x.lower()

# 약어 대체
reg_exp_replacer = RegexpReplacer()

# 단어 토큰화
def word_tokenizing(doc): # doc 단위로
    return map(nltk.word_tokenize, doc) # sent 단위로

# 문장 부호 제거
punctuation_replacer = re.compile('[%s]'%re.escape(string.punctuation))
def punctuation_replacing(doc):
    ret = []
    for sent in doc:
        sent_ret = []
        for word in sent:
            temp = punctuation_replacer.sub(u'', word)
            if not temp == u'': 
                sent_ret.append(temp)
        ret.append(sent_ret)
    return ret
    
# 불용어 제거
stop_words = set(stopwords.words('english'))
def stop_words_replacing(doc):
    ret = []
    for sent in doc:
        ret.append([word for word in sent if word not in stop_words])
    return ret

# 반복되는 문자 처리
repeat_replacer = RepeatReplacer()
def repeat_replacing(doc):
    ret=[]
    for sent in doc:
        sent_ret = []
        for word in sent:
            sent_ret.append(repeat_replacer.replace(word))
        ret.append(sent_ret)
    return ret

In [6]:
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [9]:
%%time
file_names = ['train', 'test', 'validation']
for i, df in enumerate((train, test, validation,)):
    docs = df['Text'].values.tolist()
    for foo in (lower_casing, reg_exp_replacer.replace, sent_tokenize, word_tokenizing, punctuation_replacing,repeat_replacing):
        docs = list(map(foo, docs))
    np.save('../../dataset/prepared/'+file_names[i], np.array(docs))

Wall time: 11.1 s
